# User filtering
특정 직업과 관련된 해시태그 검색 후, 해당 직업이 아닌 유저를 필터링하는 과정이 필요하다.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
# 예제 데이터
path = 'C:/Users/bogyung/Downloads/crawling/test/'
df0 = pd.read_json(path + 'tags_goonin.json')
df1 = pd.read_json(path + 'tags_gyosa.json')
df2 = pd.read_json(path + 'tags_joong2.json')
df = [df0, df1, df2]

In [3]:
print('제거 전: 군인')
display(df0.head())
print('제거 전: 교사')
display(df1.head())
print('제거 전: 중학생')
display(df2.head())

제거 전: 군인


,freq,tags,user_id,user_name
0,"{'23': 1, '24': 1}","[23, 24]",24895167311,hye_lim._.eee
1,"{'25': 5, '천안': 4, '22': 1, '군대': 1, '선임': 1, ...","[25, 천안, 22, 군대, 선임, 후임, 남자랑, 데이트, 운동, 웨이트, 군인...",4648813963,96_ugi
2,"{'셀카': 84, '맞팔': 84, '인친': 84, '소통': 84, '꿈': ...","[셀카, 맞팔, 인친, 소통, 꿈, 자부심, 팔로우, 선팔, 데일리, 여행스타그램,...",7806334927,soonshin_
3,"{'산책': 1, '여유': 1}","[산책, 여유]",37806496856,dong_bz
4,"{'여기조기요기': 1, '굿모닝': 1, '나온다나온다나이나온다': 1, '유노앤...","[여기조기요기, 굿모닝, 나온다나온다나이나온다, 유노앤드요, 컴백, 웃음욕심, 노력...",5641751675,_andyon_


제거 전: 교사


,freq,tags,user_id,user_name
0,{'범계': 1},[범계],30033400542,d_jshin
1,"{'쌤스타그램': 21, '교사스타그램': 23, '교사': 19, '선생님': 2...","[쌤스타그램, 교사스타그램, 교사, 선생님, 선생님스타그램, 중학교, 노보텔앰배서더...",3033063913,iam_minthyun
2,"{'25': 24, '쌩얼인거비밀': 1, '25살': 21, '일상': 32, '...","[25, 쌩얼인거비밀, 25살, 일상, 코로나19, 마스크필수, 어린이집, 출근, ...",2241028185,90lka11
3,"{'대구교총': 10, '교총': 6, '한국교총': 10, '이벤트': 2, '선...","[대구교총, 교총, 한국교총, 이벤트, 선생님, 교사, 교수, 유치원선생님, 쌤스타...",5571560038,cheerup_2680
4,"{'제주대학교': 12, '수시모집요강': 1, '제주대학교2021학년도수시모집요강...","[제주대학교, 수시모집요강, 제주대학교2021학년도수시모집요강, 전형일정, 모집단위...",5863131290,jejunu_ibsi


제거 전: 중학생


,freq,tags,user_id,user_name
0,"{'모트트레인9월1일차': 1, '모트트레인': 1, '고려대학교': 2, '공스타...","[모트트레인9월1일차, 모트트레인, 고려대학교, 공스타소통]",40090360824,t.memory_0
1,"{'칼단발': 1, '단발': 1, '집콕중': 1, '놀러가고싶어요': 1}","[칼단발, 단발, 집콕중, 놀러가고싶어요]",9632681759,hye0njiye0n
2,{},[],15470222343,shinchaehee06
3,"{'열공': 38, '인스': 4, '엉망진창': 1, '실수때매': 1, '망함'...","[열공, 인스, 엉망진창, 실수때매, 망함, 공부, 공스타그램, 초6, 예비중, 중...",25539217889,yeowon_study
4,"{'06': 20, '0606': 9, '공부인증그램': 2, '공부인증': 5, ...","[06, 0606, 공부인증그램, 공부인증, 공부인증샷, 공부인증계정, 공스타그램,...",27400344693,_medic.__.kim


## 검색 해시태그 제거

In [4]:
# 각 직업별 검색 해시태그 (여러 개 가능)
hashtags = [["군인"], ["교사"], ["중2"]]
N_category = 3 # 직업 카테고리 수

def del_tags(tags, hashtag):
    lst = []
    for tag in tags:
        if tag not in hashtag:
            lst.append(tag)
    return lst

def del_freq(freq, hashtag):
    dic = {}
    for (w, i) in freq.items():
        if w not in hashtag:
            dic[w] = i
    return dic

def del_hashtag(num_df, hashtag):
    df = globals()['df{}'.format(num_df)]
    tags = df.tags.apply(lambda x: del_tags(x, hashtag))
    freq = df.freq.apply(lambda x: del_freq(x, hashtag))
    df['tags'] = tags
    df['freq'] = freq
    return df

for i in range(N_category):
    globals()['df{}'.format(i)] = del_hashtag(i, hashtags[i])

In [5]:
print('제거 후: 군인')
display(df0.head())
print('제거 후: 교사')
display(df1.head())
print('제거 후: 중학생')
display(df2.head())

제거 후: 군인


,freq,tags,user_id,user_name
0,"{'23': 1, '24': 1}","[23, 24]",24895167311,hye_lim._.eee
1,"{'25': 5, '천안': 4, '22': 1, '군대': 1, '선임': 1, ...","[25, 천안, 22, 군대, 선임, 후임, 남자랑, 데이트, 운동, 웨이트, 속초...",4648813963,96_ugi
2,"{'셀카': 84, '맞팔': 84, '인친': 84, '소통': 84, '꿈': ...","[셀카, 맞팔, 인친, 소통, 꿈, 자부심, 팔로우, 선팔, 데일리, 여행스타그램,...",7806334927,soonshin_
3,"{'산책': 1, '여유': 1}","[산책, 여유]",37806496856,dong_bz
4,"{'여기조기요기': 1, '굿모닝': 1, '나온다나온다나이나온다': 1, '유노앤...","[여기조기요기, 굿모닝, 나온다나온다나이나온다, 유노앤드요, 컴백, 웃음욕심, 노력...",5641751675,_andyon_


제거 후: 교사


,freq,tags,user_id,user_name
0,{'범계': 1},[범계],30033400542,d_jshin
1,"{'쌤스타그램': 21, '교사스타그램': 23, '선생님': 22, '선생님스타그...","[쌤스타그램, 교사스타그램, 선생님, 선생님스타그램, 중학교, 노보텔앰배서더용산, ...",3033063913,iam_minthyun
2,"{'25': 24, '쌩얼인거비밀': 1, '25살': 21, '일상': 32, '...","[25, 쌩얼인거비밀, 25살, 일상, 코로나19, 마스크필수, 어린이집, 출근, ...",2241028185,90lka11
3,"{'대구교총': 10, '교총': 6, '한국교총': 10, '이벤트': 2, '선...","[대구교총, 교총, 한국교총, 이벤트, 선생님, 교수, 유치원선생님, 쌤스타그램, ...",5571560038,cheerup_2680
4,"{'제주대학교': 12, '수시모집요강': 1, '제주대학교2021학년도수시모집요강...","[제주대학교, 수시모집요강, 제주대학교2021학년도수시모집요강, 전형일정, 모집단위...",5863131290,jejunu_ibsi


제거 후: 중학생


,freq,tags,user_id,user_name
0,"{'모트트레인9월1일차': 1, '모트트레인': 1, '고려대학교': 2, '공스타...","[모트트레인9월1일차, 모트트레인, 고려대학교, 공스타소통]",40090360824,t.memory_0
1,"{'칼단발': 1, '단발': 1, '집콕중': 1, '놀러가고싶어요': 1}","[칼단발, 단발, 집콕중, 놀러가고싶어요]",9632681759,hye0njiye0n
2,{},[],15470222343,shinchaehee06
3,"{'열공': 38, '인스': 4, '엉망진창': 1, '실수때매': 1, '망함'...","[열공, 인스, 엉망진창, 실수때매, 망함, 공부, 공스타그램, 초6, 예비중, 중...",25539217889,yeowon_study
4,"{'06': 20, '0606': 9, '공부인증그램': 2, '공부인증': 5, ...","[06, 0606, 공부인증그램, 공부인증, 공부인증샷, 공부인증계정, 공스타그램,...",27400344693,_medic.__.kim


In [7]:
# 군인
freq_whole0.most_common(10)

[('일상', 7),
 ('휴가', 6),
 ('맞팔', 5),
 ('서울', 5),
 ('힐링', 5),
 ('좋아요', 5),
 ('대학생', 5),
 ('강원도', 4),
 ('코로나', 4),
 ('셀카', 4)]

## 새로운 voca dict 생성
단어의 등장 문서수가 직업군의 절반을 넘을 경우, 해당 단어를 제거한 새로운 딕셔너리 생성(각 직업군에 대표성 있다고 판단)

In [8]:
for i in range(N_category):
    globals()['voca{}'.format(i)] = {}
    globals()['freq_other{}'.format(i)] = {}

for num in range(N_category):
    for (w, n) in globals()['freq_whole{}'.format(num)].items():
        w_freq = 1
        for i in range(N_category):
            if i != num:
                if w in globals()['freq_whole{}'.format(i)]:
                    w_freq += 1
        globals()['freq_other{}'.format(num)][w] = w_freq
        if w_freq <= N_category//2:
            globals()['voca{}'.format(num)][w] = n

In [ ]:
# 각 직업의 대표성을 띠는 단어들
# voca0, voca1, voca2

# 등장한 문서 수
# freq_other0, freq_other1, freq_other2

In [9]:
N = 10 # 상위 몇 개를 뽑아낼지 지정
for i in range(N_category):
    globals()['voca{}_top'.format(i)] = dict(Counter(globals()['voca{}'.format(i)]).most_common(N))

In [10]:
# 상위 N개의 voca dict
voca0_top, voca1_top, voca2_top

({'속초': 3,
  '부사관': 3,
  '24': 2,
  '천안': 2,
  '고성': 2,
  '보내줘': 2,
  '1943': 2,
  '웃자': 2,
  '난': 2,
  '장교': 2},
 {'교사스타그램': 14,
  '쌤스타그램': 8,
  '원격수업': 6,
  '학부모': 6,
  '고등학교': 5,
  '스타벅스': 5,
  '어린이집': 4,
  '힐링타임': 4,
  '스승의날': 4,
  '책': 4},
 {'06': 12,
  '06년생': 11,
  '중2공스타그램': 8,
  '공스타그램맞팔': 7,
  '모트모트': 6,
  '모트모트플래너': 6,
  '중2공스타': 5,
  '15살': 5,
  '공부하자': 5,
  '중간피드': 5})

## negative dict 생성
- 타 직업군의 voca를 다 더한다.
- 처음에 지웠던 검색 해시태그도 추가해주기(빈도수는 df 길이, 즉 최대값으로)

In [11]:
for i in range(N_category):
    globals()['neg{}'.format(i)] = Counter()

for i in range(N_category):
    for j in range(N_category):
        if j != i:
            globals()['neg{}'.format(i)] += Counter(globals()['voca{}_top'.format(j)])

In [ ]:
# 타 직업군에서의 단어 목록
# neg0, neg1, neg2

## neg voca에 많이 등장할수록 중요도를 낮추는 방법 고찰
### negative voca에 sigmoid 적용하기
- 목표: 다른 직업군을 대표하는 단어가 등장할수록 0에 가깝고 그 외의 경우에는 1에 가깝게
- 상-하 뒤집은 후 x축으로 +N_sigmoid만큼 평행이동한 sigmoid 적용 (0~1사이 값, 변곡점은 N_sigmoid에 위치)
- N_sigmoid는 neg 단어의 빈도수 보고 조정해야 함(상위 200개의 중위값?)
- 상위 N개의 평균 값 정도가 적절할듯

In [12]:
for i in range(N_category):
    n_job = len(globals()[f'df{i}'])
    for j, tag in enumerate(hashtags):
        if j != i:
            for t in tag:
                globals()[f'neg{i}'][t] = n_job

In [13]:
def mysigmoid(x, N_sigmoid):
    return 1 / (1 + np.exp(x - N_sigmoid))

for i in range(N_category):
    globals()['neg{}_sig'.format(i)] = {}

for i in range(N_category):
    neg = globals()['neg{}'.format(i)]
    N_sigmoid = neg.most_common(N)[N//2][1] # neg 상위 N개의 중위값
    for (w, n) in neg.items():
        globals()['neg{}_sig'.format(i)][w] = mysigmoid(n, N_sigmoid)

In [ ]:
# neg0_sig, neg1_sig, neg2_sig

# scoring
### score = 평균( 해시태그 사용 빈도수 x tf-idf x neg_sig )

- 주부, 백수는 계산할 필요 없음
- 카테고리별 해시태그 단어 중요도: negative 변수를 곱한 tf-idf
- 각 유저의 해시태그 사용 총 빈도수로 나눈 값을 최종 스코어로 활용

In [14]:
for i in range(N_category):
    globals()['df{}'.format(i)]['score'] = 0

In [15]:
# 카테고리별 단어들의 tf-idf
for i in range(N_category):
    globals()['tf_idf{}'.format(i)] = {}
    for (w, n) in globals()['freq_whole{}'.format(i)].items():
        tf = n
        idf = np.log( N_category / (1 + globals()['freq_other{}'.format(i)][w]) )
        globals()['tf_idf{}'.format(i)][w] = tf * idf

In [16]:
def scoring(freq_row, num_df):
    score = 0
    N_tags = sum(freq_row.values())
    ti = globals()[f'tf_idf{num_df}']

    for (w, n) in freq_row.items():
        tf_idf = ti[w]
        try:
            neg = globals()['neg{}_sig'.format(num_df)][w]
        except:
            neg = 1
        score += tf_idf * neg / N_tags
    
    return score

In [17]:
# 실제로 돌릴때는 N_category-2 (주부, 백수)

for i in range(N_category):
    globals()['df{}'.format(i)]['score'] = globals()['df{}'.format(i)]['freq'].apply(lambda x: scoring(x, i))

# Filtering Result

### 1. 해당 직업군이 맞다.

In [18]:
print('군인')
display(df0.sort_values(by='score', ascending=False).head(10))
print('교사')
display(df1.sort_values(by='score', ascending=False).head(10))
print('중2')
display(df2.sort_values(by='score', ascending=False).head(10))

군인


,freq,tags,user_id,user_name,score
10,"{'정복': 1, '군스타그램': 1}","[정복, 군스타그램]",8523336865,nan__jinyoung,0.810930
0,"{'23': 1, '24': 1}","[23, 24]",24895167311,hye_lim._.eee,0.608198
19,"{'3년째': 1, '연애중': 1}","[3년째, 연애중]",1556311885,seol_h16_,0.405465
18,"{'창선동먹자골목': 1, '유럽여행': 1, '속옷쇼핑몰': 1, '초가을': 1...","[창선동먹자골목, 유럽여행, 속옷쇼핑몰, 초가을, 내장산, 일본야동, 만남섹스, 야...",40704112925,cvbdbcvnd,0.317865
4,"{'여기조기요기': 1, '굿모닝': 1, '나온다나온다나이나온다': 1, '유노앤...","[여기조기요기, 굿모닝, 나온다나온다나이나온다, 유노앤드요, 컴백, 웃음욕심, 노력...",5641751675,_andyon_,0.314886
12,"{'고맙다': 1, '아사도': 1, '바베큐': 2, '홀푸드마켓': 1, '갈비...","[고맙다, 아사도, 바베큐, 홀푸드마켓, 갈비, 주말, 스모크, 텍사스, 브리스킷,...",21740141066,citihunter88,0.267508
7,"{'태풍뒤맑음': 1, '러닝': 4, '딥스': 1, '싯업': 1, '난': 1...","[태풍뒤맑음, 러닝, 딥스, 싯업, 난, 넉다운, 신체검사, 말라깽이, 벌크업, 살...",6733917822,yolo_youonlyliveonce_95,0.258696
15,"{'여자숏컷': 1, '블론드': 1, '디자이너강철': 3, '미용실': 2, '...","[여자숏컷, 블론드, 디자이너강철, 미용실, 신촌미용실, 서대문구미용실, 마포구미용...",40276026268,riahn_shinchon,0.244473
21,"{'군대선물': 1, '군인스타그램': 2, '말년병장': 1, '전역전휴가': 1...","[군대선물, 군인스타그램, 말년병장, 전역전휴가, 집으로, 경찰공무원, 경찰공무원시...",2354879232,ryuji_1225__,0.235456
16,"{'리그램': 9, '생각의차이가미래를바꾼다': 5, '과거사진': 1, '챔프지점...","[리그램, 생각의차이가미래를바꾼다, 과거사진, 챔프지점장, 신라호텔, 더파크뷰, 우...",2015100987,az_one1,0.229176


교사


,freq,tags,user_id,user_name,score
20,"{'마카롱': 1, '케이크': 1, '앙샤르': 1, '입생로랑': 1, '입생로...","[마카롱, 케이크, 앙샤르, 입생로랑, 입생로랑립스틱, 스벅, 스타벅스, 스타벅스다...",2073873648,g_0on_,0.608198
7,"{'특수교사': 4, '장애': 1, '장애학': 1, '인식개선': 1, '부모교...","[특수교사, 장애, 장애학, 인식개선, 부모교육, 소통, 학급밴드, 2년차, 학부모...",25969033493,sped.learner,0.409328
25,"{'20200901': 1, '20200831': 1, '20200830': 1, ...","[20200901, 20200831, 20200830, 20200827, 20200...",27060861703,xxpeukxx,0.405465
0,{'범계': 1},[범계],30033400542,d_jshin,0.405465
4,"{'제주대학교': 12, '수시모집요강': 1, '제주대학교2021학년도수시모집요강...","[제주대학교, 수시모집요강, 제주대학교2021학년도수시모집요강, 전형일정, 모집단위...",5863131290,jejunu_ibsi,0.359126
15,"{'온라인수업': 1, '원격수업': 1, '준비': 1, '2': 1, '호텔수성...","[온라인수업, 원격수업, 준비, 2, 호텔수성, 수성호텔수영장, 헬린이, 헬린이의하...",1104816726,__j.kwang,0.293370
34,"{'싱글오리진': 1, '콜드브루': 1, '2': 4, '힐링': 5, '1': ...","[싱글오리진, 콜드브루, 2, 힐링, 1, 이게바로찐행복, 여유, 조명이다함, 많이...",1582106312,sangik_kkkkk,0.282269
9,"{'정리컨설턴트': 1, '심플라이프': 1, '미니멀라이프': 1, '신박한정리'...","[정리컨설턴트, 심플라이프, 미니멀라이프, 신박한정리, 정리방법, 수납방법, 수납꿀...",8540797781,vs_tschool,0.268618
3,"{'대구교총': 10, '교총': 6, '한국교총': 10, '이벤트': 2, '선...","[대구교총, 교총, 한국교총, 이벤트, 선생님, 교수, 유치원선생님, 쌤스타그램, ...",5571560038,cheerup_2680,0.234067
14,"{'생일선물': 1, '인스탁스': 1, '미니11': 1, '폴라로이드': 1, ...","[생일선물, 인스탁스, 미니11, 폴라로이드, 카메라, 친구가찍어준, 이벤트제조기,...",1315607116,hi_sunset.k,0.222743


중2


,freq,tags,user_id,user_name,score
9,"{'중간피드': 1, '20203월모의고사': 1}","[중간피드, 20203월모의고사]",30961379040,_lovewesxt,1.216395
5,"{'06년생': 3, '15살': 3, '전신샷': 1, '색칠스타그램': 1, '...","[06년생, 15살, 전신샷, 색칠스타그램, 소녀, 취미생활, 취미스타그램, 소확행...",40532744267,isy._.06,0.592603
8,"{'중간피드': 2, '제품제공': 2, '제브라': 1, '열공인증이벤트': 1,...","[중간피드, 제품제공, 제브라, 열공인증이벤트, 제브라방구석열공인증]",31709879503,1221_wish_,0.521312
13,"{'공스타': 4, '공스타그램맞팔': 4, '공스타맞팔': 4, '공부': 7, ...","[공스타, 공스타그램맞팔, 공스타맞팔, 공부, 공스타그램, 모트모트스터디플래너, 모...",26012283906,naeun.__.06,0.446839
6,"{'청남대': 1, '영남대학교': 1, '시골소녀': 1}","[청남대, 영남대학교, 시골소녀]",3765300236,yunji_1701,0.405465
18,{'삭제피드': 1},[삭제피드],38992930746,gmldnjss._.1,0.405465
31,"{'06': 2, '공스타그램': 5, '열공인증': 1, '사진태그': 2, '토...","[06, 공스타그램, 열공인증, 사진태그, 토이스토리, 포키, 좋아요테러, 댓글환영...",39499839360,happysh_8686,0.343607
12,"{'06': 3, '15': 2, '06년생': 1, '중딩': 1, '셀스타그램'...","[06, 15, 06년생, 중딩, 셀스타그램, 같은사람, 다른느낌, 친구들이, 좋아...",7989354612,amj0624,0.342442
4,"{'06': 20, '0606': 9, '공부인증그램': 2, '공부인증': 5, ...","[06, 0606, 공부인증그램, 공부인증, 공부인증샷, 공부인증계정, 공스타그램,...",27400344693,_medic.__.kim,0.289618
25,"{'중딩': 11, '중학생': 5, '중학교2학년': 2, '중학생2학년': 1,...","[중딩, 중학생, 중학교2학년, 중학생2학년, 중학생공부, 중학교공부, 중2공부, ...",40159949686,hyisu__x__x_06,0.232078


### 2. 해당 직업군이 아니다.

In [19]:
print('군인')
display(df0.sort_values(by='score', ascending=True).head(10))
print('교사')
display(df1.sort_values(by='score', ascending=True).head(10))
print('중2')
display(df2.sort_values(by='score', ascending=True).head(10))

군인


,freq,tags,user_id,user_name,score
3,"{'산책': 1, '여유': 1}","[산책, 여유]",37806496856,dong_bz,-0.287682
29,"{'대한민국': 1, '국방의': 1, '폭염주의보': 1, '남해': 1, '힐링...","[대한민국, 국방의, 폭염주의보, 남해, 힐링, 코로나19]",3494313245,shim9893,-0.248421
23,"{'해군': 2, '첫줄': 1, '팔로워': 1, '부사관': 1, '맞팔': 2...","[해군, 첫줄, 팔로워, 부사관, 맞팔, 선팔하면, 맞팔선팔, 맞팔해요, 소통, 대...",2204270708,hmmmm._.1120,-0.240569
30,"{'하늘': 1, '구름': 1, '여름': 1, '거울셀카': 1, '셀카놀이':...","[하늘, 구름, 여름, 거울셀카, 셀카놀이, 오늘, 오늘인척, 제주, 제주도, 호텔...",40274969790,10_yoomm_04,-0.154332
14,"{'오오티디': 2, '셀피': 7, '골프': 3, '골프선수': 6, '골프스윙...","[오오티디, 셀피, 골프, 골프선수, 골프스윙, 웨지샷, 운동선수, 대학생, 용인대...",11868515736,_hyunii____,-0.050642
20,{},[],2286824803,mj__j01,0.000000
11,{},[],23069095654,leesussam_,0.000000
22,{},[],1636081854,c.seong,0.000000
2,"{'셀카': 84, '맞팔': 84, '인친': 84, '소통': 84, '꿈': ...","[셀카, 맞팔, 인친, 소통, 꿈, 자부심, 팔로우, 선팔, 데일리, 여행스타그램,...",7806334927,soonshin_,0.011711
6,"{'그림쟁이': 137, '그림계정맞팔': 128, '그림': 178, '그림그리기...","[그림쟁이, 그림계정맞팔, 그림, 그림그리기, 그림스타그램, 일러스트레이터, 일러스...",11109557740,kkoyu123,0.015115


교사


,freq,tags,user_id,user_name,score
24,"{'여름': 1, '방학': 2, '여름방학': 2, '휴가': 2, '왜벌써': ...","[여름, 방학, 여름방학, 휴가, 왜벌써, 월요일, 주말, 돌아와, 공감, 글귀, ...",33938103350,maru_story87,-0.088658
28,{},[],31943748504,kkang.aji,0.000000
8,"{'기상인증': 35, '스터디플래너': 27, '공부기록': 32}","[기상인증, 스터디플래너, 공부기록]",34439057360,youknowjeonghyun___,0.008627
6,"{'건물주': 2, '강릉맛집': 1, '찐': 1, '찐맛집': 1, '미친맛':...","[건물주, 강릉맛집, 찐, 찐맛집, 미친맛, 이강주, 수업, 주문진, 주문진어민수산...",4776346805,2030developer,0.010509
27,"{'공스타그램': 18, '스터디플래너': 15, '플랜비': 15, '7321':...","[공스타그램, 스터디플래너, 플랜비, 7321, 열품타, 열공시간]",31813609498,s2.00000.s2,0.015017
29,"{'협찬': 12, '승무원팔뚝주사': 3, '톡스미': 3, '순플러스': 1, ...","[협찬, 승무원팔뚝주사, 톡스미, 순플러스, 순플러스민트실, 아산카페, 웜싸이트, ...",1446887286,som__bly,0.015908
12,"{'깡보람': 5, '임보람': 4, '틱톡커': 13, '틱톡': 9, '연기':...","[깡보람, 임보람, 틱톡커, 틱톡, 연기, 인스타그램, 셀스타그램, 맞팔, 선팔, ...",1815428234,forvision77_br,0.030777
13,"{'서봄': 45, '서봄어린이집': 59, '공동육아어린이집': 56, '공동육아...","[서봄, 서봄어린이집, 공동육아어린이집, 공동육아현장, 공동육아, 협동조합어린이집,...",7145448194,seobom_we_grow_together,0.057021
30,{'선생님딸내미': 7},[선생님딸내미],40034827235,teachers__daughter,0.057924
37,"{'일러스트': 21, '작품': 23, '그림': 21, '포토샵': 21, '작...","[일러스트, 작품, 그림, 포토샵, 작업, 로고, 디자인, 로고디자인, 브랜드로고,...",7862422845,jyh_0320,0.063904


중2


,freq,tags,user_id,user_name,score
16,"{'뚱뚱한마카롱': 1, '좋아요반사': 11, '인스타푸드': 1, '베프': 2...","[뚱뚱한마카롱, 좋아요반사, 인스타푸드, 베프, 19, 17, 패션, 아웃핏, 오오...",1314775199,cocoro26,-0.008471
2,{},[],15470222343,shinchaehee06,0.000000
30,{},[],40473986414,euphori_a__7,0.000000
26,{},[],8506987194,gahyeon11206,0.000000
33,{},[],11853499656,3501_oy,0.000000
24,"{'모트트레인9월1일차': 1, '공스타그램': 86, '공스타': 85, '15살...","[모트트레인9월1일차, 공스타그램, 공스타, 15살, 중2공스타그램, 중3공스타그램...",34164285683,medico_haeun,0.012251
14,"{'공스타': 26, '공부스타그램': 26, '공스타그램': 26, '공스타그램맞...","[공스타, 공부스타그램, 공스타그램, 공스타그램맞팔, 플래너, 공부자극, 공스타만맞...",33416104506,092binnn,0.033794
21,"{'외대부고캠프': 21, '용인외고캠프': 5, '외대부고': 21, '용인외고'...","[외대부고캠프, 용인외고캠프, 외대부고, 용인외고, 외대부고기숙캠프, 외대부고영어캠...",5375673573,hafscamp,0.039908
3,"{'열공': 38, '인스': 4, '엉망진창': 1, '실수때매': 1, '망함'...","[열공, 인스, 엉망진창, 실수때매, 망함, 공부, 공스타그램, 초6, 예비중, 중...",25539217889,yeowon_study,0.041169
32,"{'06': 30, '추억팔이': 2, '단발안해': 1, '장발할래': 1, '선...","[06, 추억팔이, 단발안해, 장발할래, 선팔하면맞팔, 장발, 머리기르자, 06년생...",9488010942,k_yuwon_06,0.052070


# Class
실제 데이터 적용

In [39]:
import pandas as pd
import numpy as np
from collections import Counter

class UserFiltering:
    def __init__(self, hashtags, N_category, df):
        self.hashtags = hashtags
        self.N_category = N_category
        self.df = df

    def del_tags(self, tags, hashtag):
        lst = []
        for tag in tags:
            if tag not in hashtag:
                lst.append(tag)
        return lst

    def del_freq(self, freq, hashtag):
        dic = {}
        for (w, i) in freq.items():
            if w not in hashtag:
                dic[w] = i
        return dic

    def del_hashtag(self, num_df, hashtag):
        df = self.df[num_df]
        tags = df.tags.apply(lambda x: self.del_tags(x, hashtag))
        freq = df.freq.apply(lambda x: self.del_freq(x, hashtag))
        df['tags'] = tags
        df['freq'] = freq
        return df
    
    # 1) 검색 해시태그 제거
    def delete_hashtag(self):
        df = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            df[i] = self.del_hashtag(i, self.hashtags[i])
        return df

    def filtering(self, N): # N: 각 직업을 대표하는 voca에서 상위 몇 개를 뽑아낼지
        df = self.delete_hashtag()

        # 2) unique tags의 빈도수
        freq_whole = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            tmp = []
            for t in df[i].tags:
                tmp.extend(t)
            freq_whole[i] = Counter(tmp)

        # 3) 단어의 등장 문서수가 직업군의 절반을 넘을 경우, 해당 단어를 제거한 새로운 voca dictionary 생성
        voca = [0 for i in range(self.N_category)]
        freq_other = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            voca[i] = {}
            freq_other[i] = {}

        for num in range(self.N_category):
            for (w, n) in freq_whole[num].items():
                w_freq = 1
                for i in range(self.N_category):
                    if i != num:
                        if w in freq_whole[i]:
                            w_freq += 1
                freq_other[num][w] = w_freq
                if w_freq <= self.N_category//2:
                    voca[num][w] = n
        
        voca_top = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            voca_top[i] = dict(Counter(voca[i]).most_common(N))

        # 4) negative dictionary 생성: 타 직업군을 대표하는 단어
        neg = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            neg[i] = Counter()

        for i in range(self.N_category):
            for j in range(self.N_category):
                if j != i:
                    neg[i] += Counter(voca_top[j])

        # 5) negative voca에 변형된 sigmoid 적용: neg voca에 많이 등장할수록 중요도를 낮춤
        for i in range(self.N_category):
            n_job = len(df[i])
            for j, tag in enumerate(self.hashtags):
                if j != i:
                    for t in tag:
                        neg[i][t] = n_job
        
        neg_sig = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            neg_sig[i] = {}

        for i in range(self.N_category):
            nega = neg[i]
            point = nega.most_common(N)[N//2][1] # sigmoid 변곡점: neg 상위 N개의 중위값에 해당하는 빈도수
            for (w, n) in nega.items():
                neg_sig[i][w] = 1 / (1 + np.exp(n) - point)

        # 6) scoring: score = 평균(해시태그 사용 빈도수 x tf-idf x neg_sig)
        for i in range(self.N_category):
            df[i]['score'] = 0

        tf_idf = [0 for i in range(self.N_category)]
        for i in range(self.N_category):
            tf_idf[i] = {}
            for (w, n) in freq_whole[i].items():
                tf = n
                idf = np.log( self.N_category / (1 + freq_other[i][w]) )
                tf_idf[i][w] = tf * idf
        
        def scoring(freq_row, num_df):
            score = 0
            N_tags = sum(freq_row.values())
            ti = tf_idf[num_df]

            for (w, n) in freq_row.items():
                tf_idf_score = ti[w]
                try:
                    nega = neg_sig[num_df][w]
                except:
                    nega = 1
                score += tf_idf_score * nega / N_tags
            
            return score

        for i in range(self.N_category-2): # 주부, 백수 제외
            df[i]['score'] = df[i]['freq'].apply(lambda x: scoring(x, i))

        return df, voca_top

In [36]:
N_category=9
hashtags = [
    ["간호사", "나이팅게일선서식", "널스타그램", "응급실간호사", "간호사그램"],
    ["경찰", "경찰스타그램", "중앙경찰학교"],
    ["교사", "교사스타그램", "쌤스타그램"],
    ["군인", "군스타그램", "군인스타그램"],
    ["소방관", "소방공무원", "소방스타그램"],
    ["의사", "의사스타그램"],
    ["고1", "고2", "고3", "중1", "중2", "중3"],
    ["딸맘그램"],
    ["백수"]
]

path = 'C:/Users/bogyung/Downloads/crawling/test/'
df = [0 for i in range(N_category)]

uf = UserFiltering(hashtags=hashtags, N_category=3, df=df)